In [1]:
import numpy as np
import sys
import os
from pathlib import Path
notebook_path = Path(os.getcwd())
sys.path.append(str(notebook_path.parent))
import scale.olm as olm
import subprocess

2023-09-01 22:10:28 [info     ] Initialized scale.olm


In [4]:
# Retrieve a single page and report the URL and contents
def run_command(id, command_line, check_return_code=True, echo=True, error_match="Error"):
    """Run a command as a subprocess. Throw on bad error code or finding 'Error' in the output."""

    p = subprocess.Popen(
        command_line,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True,
        encoding="utf-8",
    )

    text = ""
    while True:
        line = p.stdout.readline()
        text += line
        if error_match in line:
            raise ValueError(line.strip())
        if not line:
            break

    if not p.returncode:
        retcode = 1
    else:
        retcode = p.returncode

    if retcode != 0 and text.strip() == "":
        raise ValueError(
            f"command line='{command_line}' failed to run in the shell. Check this is a valid path or recognized executable."
        )
    elif check_return_code and retcode < 0:
        msg = p.stderr.read().strip()
        if not msg == "":
            raise ValueError(str(msg))
    return (id,text)

In [5]:
import concurrent.futures
from tqdm import tqdm
import random

commands=[]
for r in range(100):
    z=random.randint(4, 7)
    commands.append("sleep {}".format(z,z))

# We can use a with statement to ensure threads are cleaned up promptly
with tqdm(total=len(commands)) as pbar:
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        # Start the load operations and mark each future with its URL
        submits = {executor.submit(run_command,id,commands[id],echo=False): id for id in range(len(commands))}
        results = {}
        for future in concurrent.futures.as_completed(submits):
            arg = submits[future]
            results[arg]= future.result()
            pbar.update(1)
print(data)

  0%|                                                                                                        | 0/100 [00:26<?, ?it/s]


KeyboardInterrupt: 